In [1]:
import os
import torch
from diffusers import DiffusionPipeline, AutoencoderKL
from random import choice
from tqdm import tqdm
import shutil

print(f"CUDA is available: {torch.cuda.is_available()}")
print(f"Amount of CUDA devices available: {torch.cuda.device_count()}")
print(f"Index of current CUDA device: {torch.cuda.current_device()}")
print(f"Name of current CUDA device: {torch.cuda.get_device_name(torch.cuda.current_device())}")
print(f"Amount of CPU cores available: {os.cpu_count()}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cores = os.cpu_count()

/home/olejohanjo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA is available: True
Amount of CUDA devices available: 1
Index of current CUDA device: 0
Name of current CUDA device: Tesla V100-SXM3-32GB
Amount of CPU cores available: 96


In [2]:
categories = [
    "village",
    "vineyard",
    "nursery",
    "banquet_hall",
    "beach",
    "kindergarden_classroom",
    "elevator_lobby",
    "moat-water",
    "islet",
    "shopfront"
]

prompts_per_category = {
    "village": [
        "a realistic photo of a peaceful village in the countryside",
        "a realistic photo of a quaint village with cobblestone streets",
        "a realistic photo of a village nestled between rolling hills",
        "a realistic photo of a village square during a sunny day",
        "a realistic photo of a traditional village with wooden houses",
        "a realistic photo of a village market bustling with people",
        "a realistic photo of a village at sunset with golden hues",
        "a realistic photo of a small village with narrow streets",
        "a realistic photo of a charming village with gardens and flowers",
        "a realistic photo of a quiet village covered in snow",
        "a realistic photo of a village near a river with boats docked",
        "a realistic photo of a rural village with farm animals",
        "a realistic photo of a village surrounded by lush green fields",
        "a realistic photo of a medieval village with stone houses",
        "a realistic photo of a village with thatched-roof cottages",
        "a realistic photo of a serene village during autumn",
        "a realistic photo of a village with a church steeple",
        "a realistic photo of a lively village fair with colorful decorations",
        "a realistic photo of a deserted village road on a foggy morning",
        "a realistic photo of a village at dawn with glowing lights"
    ],
    "vineyard": [
        "a realistic photo of a vineyard during harvest season",
        "a realistic photo of a lush vineyard stretching over hills",
        "a realistic photo of a vineyard with rows of grapevines",
        "a realistic photo of a vineyard at sunset with a view of the hills",
        "a realistic photo of grapes ripening on the vine in a vineyard",
        "a realistic photo of a vineyard with workers picking grapes",
        "a realistic photo of a vineyard in autumn with golden leaves",
        "a realistic photo of a vineyard on a misty morning",
        "a realistic photo of a vineyard with a rustic farmhouse",
        "a realistic photo of a vineyard with wine barrels stacked nearby",
        "a realistic photo of a vineyard overlooking a serene valley",
        "a realistic photo of a vineyard during spring with blooming flowers",
        "a realistic photo of a vineyard with trellises full of grape clusters",
        "a realistic photo of a vineyard with a tasting area for wine",
        "a realistic photo of a vineyard on a cloudy day with dramatic skies",
        "a realistic photo of a small vineyard surrounded by mountains",
        "a realistic photo of a sprawling vineyard under a bright blue sky",
        "a realistic photo of a vineyard with rows perfectly aligned",
        "a realistic photo of a vineyard with a stone pathway through it",
        "a realistic photo of a vineyard illuminated by the morning sun"
    ],
    "nursery": [
        "a realistic photo of a child’s nursery decorated with pastel colors",
        "a realistic photo of a nursery with soft lighting and plush toys",
        "a realistic photo of a cozy nursery with a crib and rocking chair",
        "a realistic photo of a modern nursery with neutral tones",
        "a realistic photo of a nursery filled with bright, cheerful colors",
        "a realistic photo of a nursery with a mobile hanging above a crib",
        "a realistic photo of a nursery with stuffed animals on a shelf",
        "a realistic photo of a nursery with a playmat on the floor",
        "a realistic photo of a nursery with starry night wallpaper",
        "a realistic photo of a minimalist nursery with clean lines",
        "a realistic photo of a twin nursery with matching cribs",
        "a realistic photo of a spacious nursery with an armchair for parents",
        "a realistic photo of a nursery with books stacked on a small shelf",
        "a realistic photo of a nursery with soft rugs and warm lighting",
        "a realistic photo of a nursery with a baby monitor on the side table",
        "a realistic photo of a nursery with colorful wall decals",
        "a realistic photo of a nursery decorated for a newborn baby",
        "a realistic photo of a nursery with a chalkboard wall for drawing",
        "a realistic photo of a nursery with a small rocking horse",
        "a realistic photo of a nursery with a dresser and changing table"
    ],
    "banquet_hall": [
        "a realistic photo of a grand banquet hall decorated for a wedding",
        "a realistic photo of an elegant banquet hall with chandeliers",
        "a realistic photo of a banquet hall set up for a formal dinner",
        "a realistic photo of a spacious banquet hall with long tables",
        "a realistic photo of a luxurious banquet hall with marble floors",
        "a realistic photo of a banquet hall with golden drapes and floral arrangements",
        "a realistic photo of a banquet hall during a corporate event",
        "a realistic photo of a banquet hall with round tables and elegant centerpieces",
        "a realistic photo of a banquet hall with a stage for performances",
        "a realistic photo of a banquet hall with high ceilings and large windows",
        "a realistic photo of a banquet hall illuminated by soft warm lighting",
        "a realistic photo of a banquet hall prepared for a gala dinner",
        "a realistic photo of a banquet hall with modern, minimalist decor",
        "a realistic photo of a banquet hall adorned with fairy lights and candles",
        "a realistic photo of a banquet hall with a large buffet table",
        "a realistic photo of a banquet hall with ornate pillars and a red carpet",
        "a realistic photo of a banquet hall with tables arranged in a U-shape",
        "a realistic photo of a banquet hall hosting a cultural festival",
        "a realistic photo of a banquet hall with a dance floor in the center",
        "a realistic photo of a banquet hall with luxurious velvet seating"
    ],
    "beach": [
        "a realistic photo of a sunny beach with palm trees",
        "a realistic photo of a beach during sunset with waves crashing",
        "a realistic photo of an aerial view of a tropical beach",
        "a realistic photo of a crowded beach on a sunny day",
        "a realistic photo of a serene beach with clear blue water",
        "a realistic photo of a beach with white sand and turquoise waters",
        "a realistic photo of children playing with sandcastles on a beach",
        "a realistic photo of a beach with colorful umbrellas and sunbeds",
        "a realistic photo of a secluded beach with rocky cliffs",
        "a realistic photo of a beach at sunrise with pink and orange skies",
        "a realistic photo of surfers riding waves at a beach",
        "a realistic photo of a beach with sea turtles resting on the shore",
        "a realistic photo of a beach surrounded by lush green hills",
        "a realistic photo of a quiet beach with driftwood and seashells",
        "a realistic photo of a beach with a lifeguard tower and flags",
        "a realistic photo of a tropical beach with hammocks tied to trees",
        "a realistic photo of a beach during a storm with dramatic clouds",
        "a realistic photo of a beach at night under a starry sky",
        "a realistic photo of a beach with fishing boats docked near the shore",
        "a realistic photo of a beach with a pier extending into the ocean"
    ],
    "kindergarden_classroom": [
        "a realistic photo of a colorful kindergarden classroom with toys",
        "a realistic photo of a bright and cheerful kindergarden classroom",
        "a realistic photo of children playing in a kindergarden classroom",
        "a realistic photo of a kindergarden classroom with small desks and chairs",
        "a realistic photo of a kindergarden classroom decorated with animal posters",
        "a realistic photo of a kindergarden classroom with art supplies on tables",
        "a realistic photo of a kindergarden classroom with an interactive whiteboard",
        "a realistic photo of a kindergarden classroom with alphabet charts on the walls",
        "a realistic photo of a kindergarden classroom with a reading corner filled with books",
        "a realistic photo of a kindergarden classroom with colorful floor mats",
        "a realistic photo of a kindergarden classroom during storytime with kids sitting in a circle",
        "a realistic photo of a kindergarden classroom with paintings and crafts displayed on walls",
        "a realistic photo of a kindergarden classroom with musical instruments for children",
        "a realistic photo of a kindergarden classroom with a teacher guiding children at a table",
        "a realistic photo of a kindergarden classroom with a play area and building blocks",
        "a realistic photo of a kindergarden classroom with children drawing and coloring",
        "a realistic photo of a kindergarden classroom with seasonal decorations on the windows",
        "a realistic photo of a kindergarden classroom with cubbies for children’s belongings",
        "a realistic photo of a kindergarden classroom with educational posters and charts",
        "a realistic photo of a kindergarden classroom with kids happily engaging in group activities"
    ],
    "elevator_lobby": [
        "a realistic photo of a modern elevator lobby with sleek walls",
        "a realistic photo of a luxurious elevator lobby with marble floors",
        "a realistic photo of an elevator lobby with golden accents and mirrors",
        "a realistic photo of a busy elevator lobby in a skyscraper",
        "a realistic photo of an elevator lobby with a panoramic view of the city",
        "a realistic photo of an elevator lobby with wooden paneling and soft lighting",
        "a realistic photo of an elevator lobby with digital display panels",
        "a realistic photo of an elevator lobby in a hotel with ornate decorations",
        "a realistic photo of an elevator lobby with plush seating areas",
        "a realistic photo of an elevator lobby with glass walls overlooking a garden",
        "a realistic photo of an elevator lobby in an office building with a minimalist design",
        "a realistic photo of an elevator lobby with an artistic ceiling installation",
        "a realistic photo of an elevator lobby with plants and greenery for decoration",
        "a realistic photo of an elevator lobby with high ceilings and chandeliers",
        "a realistic photo of an elevator lobby in a residential apartment complex",
        "a realistic photo of an elevator lobby with polished stainless steel doors",
        "a realistic photo of an elevator lobby during the daytime with natural sunlight",
        "a realistic photo of an elevator lobby in a shopping mall with advertisements",
        "a realistic photo of an elevator lobby with emergency stairs visible nearby",
        "a realistic photo of an elevator lobby with a concierge desk in the corner"
    ],
    "moat-water": [
        "a realistic photo of a medieval castle surrounded by a moat",
        "a realistic photo of a peaceful moat with calm water and reflections",
        "a realistic photo of a moat with water lilies floating on the surface",
        "a realistic photo of a moat reflecting the surrounding castle walls",
        "a realistic photo of a clear moat with fish swimming beneath the surface",
        "a realistic photo of a moat with a wooden bridge crossing it",
        "a realistic photo of a moat with tall grass growing along the edges",
        "a realistic photo of a moat during autumn with colorful leaves floating",
        "a realistic photo of a moat encircling a small fortress",
        "a realistic photo of a moat with ducks swimming in the water",
        "a realistic photo of a nighttime view of a moat with castle lights reflecting",
        "a realistic photo of a wide moat filled with crystal-clear water",
        "a realistic photo of a moat with a drawbridge raised above it",
        "a realistic photo of a moat surrounded by trees and wildflowers",
        "a realistic photo of a dry moat showing its historic stone walls",
        "a realistic photo of a shallow moat with stepping stones crossing it",
        "a realistic photo of a moat with a swan gliding gracefully across the water",
        "a realistic photo of a vibrant moat with aquatic plants growing inside",
        "a realistic photo of a calm moat with fog rolling in during the early morning",
        "a realistic photo of a wide, deep moat with tall stone walls on both sides"
    ],
    "islet": [
        "a realistic photo of a small islet in the middle of a serene lake",
        "a realistic photo of an islet surrounded by crystal-clear ocean water",
        "a realistic photo of a tropical islet with palm trees and sandy shores",
        "a realistic photo of a rocky islet with waves crashing against it",
        "a realistic photo of a secluded islet with vibrant green vegetation",
        "a realistic photo of an islet at sunset with golden light reflecting on the water",
        "a realistic photo of a tiny islet with a single tree in the center",
        "a realistic photo of an islet with birds nesting along the rocky edges",
        "a realistic photo of a group of islets scattered across a turquoise lagoon",
        "a realistic photo of an islet surrounded by calm water and distant mountains",
        "a realistic photo of an islet with a small wooden cabin on it",
        "a realistic photo of an islet with white sand beaches and coral reefs nearby",
        "a realistic photo of an islet with mist hovering above the water in the early morning",
        "a realistic photo of an islet with a small lighthouse standing tall",
        "a realistic photo of an islet in a fjord surrounded by towering cliffs",
        "a realistic photo of a lush islet with flowers blooming on its shores",
        "a realistic photo of an islet with a dock and a small fishing boat tied up",
        "a realistic photo of an islet with clear water showing colorful fish below",
        "a realistic photo of a barren islet with jagged rocks and no vegetation",
        "a realistic photo of an islet under a starry night sky with water shimmering around it"
    ],
    "shopfront": [
        "a realistic photo of a charming shopfront with colorful window displays",
        "a realistic photo of a modern shopfront with sleek glass walls",
        "a realistic photo of a vintage shopfront with wooden signs and rustic decor",
        "a realistic photo of a bustling shopfront in a busy market street",
        "a realistic photo of a stylish shopfront with a minimalist design",
        "a realistic photo of a flower shopfront with vibrant bouquets on display",
        "a realistic photo of a small bakery shopfront with pastries visible through the window",
        "a realistic photo of a clothing shopfront with mannequins in the display",
        "a realistic photo of a bookshop shopfront with a cozy and inviting entrance",
        "a realistic photo of a cafe shopfront with outdoor seating and a chalkboard menu",
        "a realistic photo of a shopfront decorated with festive holiday lights",
        "a realistic photo of a quaint antique shopfront with unique items in the window",
        "a realistic photo of a boutique shopfront with elegant gold lettering on the glass",
        "a realistic photo of a shopfront in a historic building with detailed architecture",
        "a realistic photo of a street with multiple colorful shopfronts side by side",
        "a realistic photo of a shopfront selling handmade crafts with a rustic vibe",
        "a realistic photo of a busy shopfront with people walking by during the daytime",
        "a realistic photo of a shopfront with vibrant murals painted on the walls",
        "a realistic photo of a small grocery shopfront with fresh produce outside",
        "a realistic photo of a shopfront at night with bright neon signs glowing"
    ]
}


default_prompts = {
    category: [f"A realistic photo of a {category}"] for category in categories if category not in prompts_per_category
}
prompts_per_category.update(default_prompts)

In [3]:
vae = AutoencoderKL.from_pretrained(
    "madebyollin/sdxl-vae-fp16-fix",
    torch_dtype=torch.float16
)

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
).to("cuda")
pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)

refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
).to("cuda")
refiner.unet = torch.compile(refiner.unet, mode="reduce-overhead", fullgraph=True)


output_dir = "generated_dataset"
os.makedirs(output_dir, exist_ok=True)

num_images_per_category = 1000
batch_size = 3
num_inference_steps = 24

Loading pipeline components...: 100%|██████████| 5/5 [00:00<00:00,  6.63it/s]


In [ ]:
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    print("Deleted existing directory.")

for category in categories:
    category_dir = os.path.join(output_dir, category)
    os.makedirs(category_dir, exist_ok=True)

    prompts = prompts_per_category[category]
    num_batches = num_images_per_category // batch_size

    print(f"Generating images for category: {category}")
    for batch_idx in tqdm(range(num_batches), desc=f"Category: {category}"):
        batch_prompts = [choice(prompts) for _ in range(batch_size)]
        generator = torch.Generator(device='cuda')
        seeds = [torch.seed() for _ in range(batch_size)]  # Generate random seeds

        with torch.autocast("cuda", dtype=torch.float16):
            # Generate images with the base model
            base_images = pipe(
                prompt=batch_prompts,
                generator=[generator.manual_seed(seed) for seed in seeds],
                num_inference_steps=num_inference_steps,
            ).images

            # Refine the images
            images = refiner(
                prompt=batch_prompts,
                image=base_images,
                generator=[generator.manual_seed(seed) for seed in seeds],
                num_inference_steps=num_inference_steps,
            ).images

        # Save the images
        for i, image in enumerate(images):
            image_idx = batch_idx * batch_size + i
            image_filename = os.path.join(category_dir, f"{category}_{image_idx:05d}.png")
            image.save(image_filename)
            #print(f"Saved {image_filename} with prompt: '{batch_prompts[i]}' and seed: {seeds[i]}")

Deleted existing directory.
Generating images for category: village


100%|██████████| 24/24 [02:07<00:00,  5.30s/it]

  0%|          | 0/7 [00:00<?, ?it/s]